In [1]:
# Automatic reload imports
%load_ext autoreload
%autoreload 2

In [2]:
# Run notebooks as it would run if was one folder upper in the tree, enabling direct reference to source files
import sys
sys.path.insert(0, '../')

In [3]:
import torch
import torch.nn as nn
import numpy as np

from src.utils.level_dict import hierarchy
from src.model.hierarchical_loss import HierarchicalLossNetwork

In [4]:
if torch.cuda.is_available():
        available_device = "cuda" 
else:
    available_device = 'cpu'
device = torch.device(available_device)

print(f'{torch.cuda.device_count()} GPUs available.')
print(f'Torch is using {device}!')

2 GPUs available.
Torch is using cuda!


In [5]:
hloss = HierarchicalLossNetwork(
    metafile_path='../dataset/pickle_files/meta',
    hierarchical_labels=hierarchy,
    device=device,
    p_loss=3,
    alpha=1,
    beta=1
)

In [6]:
hloss.numeric_hierarchy

{0: [4, 30, 55, 72, 95],
 1: [1, 32, 67, 73, 91],
 2: [54, 62, 70, 82, 92],
 3: [9, 10, 16, 28, 61],
 4: [0, 51, 53, 57, 83],
 5: [22, 39, 40, 86, 87],
 6: [5, 20, 25, 84, 94],
 7: [6, 7, 14, 18, 24],
 8: [3, 42, 43, 88, 97],
 9: [12, 17, 37, 68, 76],
 10: [23, 33, 49, 60, 71],
 11: [15, 19, 21, 31, 38],
 12: [34, 63, 64, 66, 75],
 13: [26, 45, 77, 79, 99],
 14: [2, 11, 35, 46, 98],
 15: [27, 29, 44, 78, 93],
 16: [36, 50, 65, 74, 80],
 17: [47, 52, 56, 59, 96],
 18: [8, 13, 48, 58, 90],
 19: [41, 69, 81, 85, 89]}

In [11]:
def make_fake_predictions(predictions_index, numeric_hierarchy, device):
    n_classes = {}
    n_classes['superclass'] = len(list(numeric_hierarchy.keys()))
    n_classes['subclass'] = len(np.ravel(list(hloss.numeric_hierarchy.values())))

    fake_predictions = {}
    fake_predictions['superclass'] = []
    fake_predictions['subclass'] = []

    for class_type in ['superclass', 'subclass']:
        for example in predictions_index:
            class_index = example[0 if class_type=='superclass' else 1]
            fake_prediction = np.zeros(n_classes[class_type])
            fake_prediction[class_index] = 1

            fake_predictions[class_type].append(fake_prediction)
        
        
        fake_predictions[class_type] = torch.Tensor(fake_predictions[class_type])
        fake_predictions[class_type] = fake_predictions[class_type].to(device)

    return [fake_predictions['superclass'], fake_predictions['subclass']]

def make_fake_labels(labels, device):
    tensor_labels = torch.Tensor(labels).to(device)
    tensor_labels = [tensor_labels[:,0], tensor_labels[:,1]]
    return tensor_labels

In [26]:
predictions_index = [
    [0, 55],  # Acerto; acertou superclasse, acertou subclasse, acertou dependencia
    [0, 30],  # Erro;   acertou superclasse, errou   subclasse, acertou dependencia
    [0, 32],  # Erro;   acertou superclasse, errou   subclasse, errou   dependencia
    [1, 55],  # Erro;   errou   superclasse, acertou subclasse, errou   dependencia
    [1, 32],  # Erro;   errou   superclasse, errou   subclasse, acertou dependencia
    [1, 62],  # Erro;   errou   superclasse, errou   subclasse, errou   dependencia
              # Erro;   acertou superclasse, acertou subclasse, errou   dependencia <- impossivel pois se acertar os dois acerta a dependencia
              # Erro;   errou   superclasse, acertou subclasse, acertou dependencia <- impossivel
]
labels = [
    [0, 55],
    [0, 55],
    [0, 55],
    [0, 55],
    [0, 55],
    [0, 55],
]

error_type = [
    'Acerto; acertou superclasse, acertou subclasse, acertou dependencia',
    'Erro;   acertou superclasse, errou   subclasse, acertou dependencia',
    'Erro;   acertou superclasse, errou   subclasse, errou   dependencia',
    'Erro;   errou   superclasse, acertou subclasse, errou   dependencia',
    'Erro;   errou   superclasse, errou   subclasse, acertou dependencia',
    'Erro;   errou   superclasse, errou   subclasse, errou   dependencia'
]

In [13]:
fake_predictions = make_fake_predictions(predictions_index, hloss.numeric_hierarchy, device)
fake_predictions

[tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.]], device='cuda:0'),
 tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [14]:
superclasses_predictions = torch.argmax(nn.Softmax(dim=1)(fake_predictions[0]), dim=1)
sublcasses_predictions = torch.argmax(nn.Softmax(dim=1)(fake_predictions[1]), dim=1)

print(f'superclasses_predictions: {superclasses_predictions}')
print(f'sublcasses_predictions: {sublcasses_predictions}')

fake_labels = make_fake_labels(labels, device)
print(f'fake_labels: {fake_labels}')

superclasses_predictions: tensor([0, 0, 0, 1, 1, 1], device='cuda:0')
sublcasses_predictions: tensor([55, 30, 32, 55, 32, 62], device='cuda:0')
fake_labels: [tensor([0., 0., 0., 0., 0., 0.], device='cuda:0'), tensor([55., 55., 55., 55., 55., 55.], device='cuda:0')]


In [15]:
hloss.calculate_lloss(fake_predictions, fake_labels)

RuntimeError: "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Float'

In [20]:
# l_loss = hloss.calculate_lloss(fake_predictions, fake_labels)
# print(f'LLoss: {lloss}')

d_loss = hloss.calculate_dloss(fake_predictions, fake_labels)
d_loss


tensor(12., device='cuda:0')

In [30]:
for i in range(len(predictions_index)):
    fake_predictions = make_fake_predictions([predictions_index[i]], hloss.numeric_hierarchy, device)
    fake_labels = make_fake_labels([labels[i]], device)

    d_loss = hloss.calculate_dloss(fake_predictions, fake_labels)
    print("#", error_type[i])
    # print(f'label: {labels[i]} | prediction: {predictions_index[i]}')
    print(f'Dloss: {d_loss}')


# Acerto; acertou superclasse, acertou subclasse, acertou dependencia
Dloss: 0.0
# Erro;   acertou superclasse, errou   subclasse, acertou dependencia
Dloss: 0.0
# Erro;   acertou superclasse, errou   subclasse, errou   dependencia
Dloss: 2.0
# Erro;   errou   superclasse, acertou subclasse, errou   dependencia
Dloss: 2.0
# Erro;   errou   superclasse, errou   subclasse, acertou dependencia
Dloss: 0.0
# Erro;   errou   superclasse, errou   subclasse, errou   dependencia
Dloss: 8.0


In [22]:
predictions_index[0]

[0, 55]